In [67]:
import pandas as pd
import os
import re

folder_path = 'filtered_data'
dfs = []

print("🗂️ Читання файлів...\n")

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        print(f"🔍 Файл: {file}")
        
        # Витягнути рік із назви файлу
        year_match = re.search(r'(\d{4})', file)
        if not year_match:
            print("⚠️  Пропущено: не знайдено рік у назві")
            continue
        year = int(year_match.group(1))
        print(f"📅 Рік: {year}")
        
        # Визначити тип іспиту
        source = 'ZNO' if year < 2022 else 'EIT'
        print(f"📌 Тип іспиту: {source}")
        
        # Зчитати CSV
        path = os.path.join(folder_path, file)
        df = pd.read_csv(path)
        print(f"✅ Прочитано: {len(df)} рядків")

        # Фільтрація по регіону залежно від source
        if source == 'ZNO':
            # Для ZNO перевіряємо наявність math_pt_region (або інших pt_region)
            pt_col_candidates = ['math_pt_region', 'pt_region_name']
            pt_col = None
            for col in pt_col_candidates:
                if col in df.columns:
                    pt_col = col
                    break
            
            if pt_col:
                before_filter = len(df)
                df = df[df[pt_col] == df['region_name']]
                after_filter = len(df)
                print(f"⚙️ Фільтр {pt_col} == region_name: рядків було {before_filter}, стало {after_filter} (обрізано {before_filter - after_filter})")
            else:
                print(f"⚠️ Не знайдено колонок {pt_col_candidates} для фільтрації у ZNO")
        
        else:  # source == 'EIT'
            if 'pt_region_name' in df.columns and 'region_name' in df.columns:
                before_filter = len(df)
                df = df[df['pt_region_name'] == df['region_name']]
                after_filter = len(df)
                print(f"⚙️ Фільтр pt_region_name == region_name: рядків було {before_filter}, стало {after_filter} (обрізано {before_filter - after_filter})")
            else:
                print("⚠️ У файлі немає колонок 'pt_region_name' і 'region_name' для фільтрації")

        # Перевірка колонок для подальшої агрегації
        if 'region_name' in df.columns and 'average_score' in df.columns:
            print("✔️  Знайдено потрібні колонки")
            df = df.groupby('region_name', as_index=False)['average_score'].mean()
            df['year'] = year
            df['source'] = source
            dfs.append(df)
        else:
            print("❌ Немає колонок 'region_name' і 'average_score', пропущено\n")

print("\n📊 Об'єднання даних...")

# Об'єднати всі файли в один DataFrame
data = pd.concat(dfs, ignore_index=True)
print(f"🔗 Усього рядків: {len(data)}")
print(data.head())

# Статистика по source + year
print("\n📈 Статистика average_score по роках і типах тестів:")
print(data.groupby(['source', 'year'])['average_score'].describe())

# Мін-макс нормалізація всередині source + year
print("\n⚙️ Обчислення score_norm...")
data['score_norm'] = data.groupby(['source', 'year'])['average_score'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

# Відносний індекс у межах source + year з середнім
print("⚙️ Обчислення score_rel...")
data['score_rel'] = data['average_score'] / data.groupby(['source', 'year'])['average_score'].transform('mean')

# Результат
print("\n✅ Результат (топ 20):")
print(data.sort_values(['source', 'year', 'score_norm'], ascending=[True, True, False]).head(20))


🗂️ Читання файлів...

🔍 Файл: 2020.csv
📅 Рік: 2020
📌 Тип іспиту: ZNO


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/4284509160.py:28: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Прочитано: 201212 рядків
⚙️ Фільтр math_pt_region == region_name: рядків було 201212, стало 118084 (обрізано 83128)
✔️  Знайдено потрібні колонки
🔍 Файл: 2021.csv
📅 Рік: 2021
📌 Тип іспиту: ZNO


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/4284509160.py:28: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Прочитано: 188609 рядків
⚙️ Фільтр math_pt_region == region_name: рядків було 188609, стало 164486 (обрізано 24123)
✔️  Знайдено потрібні колонки
🔍 Файл: 2023.csv
📅 Рік: 2023
📌 Тип іспиту: EIT
✅ Прочитано: 256313 рядків
⚙️ Фільтр pt_region_name == region_name: рядків було 256313, стало 220897 (обрізано 35416)
✔️  Знайдено потрібні колонки
🔍 Файл: 2022.csv
📅 Рік: 2022
📌 Тип іспиту: EIT


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/4284509160.py:28: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Прочитано: 213647 рядків
⚙️ Фільтр pt_region_name == region_name: рядків було 213647, стало 172865 (обрізано 40782)
✔️  Знайдено потрібні колонки
🔍 Файл: 2019.csv
📅 Рік: 2019
📌 Тип іспиту: ZNO


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/4284509160.py:28: DtypeWarning: Columns (100,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


✅ Прочитано: 172734 рядків
⚙️ Фільтр math_pt_region == region_name: рядків було 172734, стало 96606 (обрізано 76128)
✔️  Знайдено потрібні колонки
🔍 Файл: 2024.csv
📅 Рік: 2024
📌 Тип іспиту: EIT
✅ Прочитано: 264164 рядків
⚙️ Фільтр pt_region_name == region_name: рядків було 264164, стало 235488 (обрізано 28676)
✔️  Знайдено потрібні колонки

📊 Об'єднання даних...
🔗 Усього рядків: 141
                 region_name  average_score  year source
0  Івано-Франківська область     149.334603  2020    ZNO
1          Волинська область     148.794666  2020    ZNO
2          Вінницька область     144.970217  2020    ZNO
3   Дніпропетровська область     145.102046  2020    ZNO
4           Донецька область     143.967670  2020    ZNO

📈 Статистика average_score по роках і типах тестів:
             count        mean       std         min         25%         50%  \
source year                                                                    
EIT    2022   22.0  150.437761  2.094032  147.029111  149.1

In [68]:
data

,region_name,average_score,year,source,score_norm,score_rel
0,Івано-Франківська область,149.334603,2020,ZNO,0.578887,1.019871
1,Волинська область,148.794666,2020,ZNO,0.531764,1.016183
2,Вінницька область,144.970217,2020,ZNO,0.197989,0.990065
3,Дніпропетровська область,145.102046,2020,ZNO,0.209494,0.990965
4,Донецька область,143.967670,2020,ZNO,0.110493,0.983218
...,...,...,...,...,...,...
136,Хмельницька область,139.375244,2024,EIT,0.285166,1.002219
137,Черкаська область,138.331308,2024,EIT,0.147698,0.994712
138,Чернівецька область,138.119851,2024,EIT,0.119853,0.993192
139,Чернігівська область,138.846084,2024,EIT,0.215485,0.998414


In [63]:
summary = data.groupby(['region_name', 'source']).agg(
    average_score_mean=('average_score', 'mean'),
    score_norm_mean=('score_norm', 'mean'),
    score_rel_mean=('score_rel', 'mean'),
    count_years=('year', 'nunique')
).reset_index()

summary


,region_name,source,average_score_mean,score_norm_mean,score_rel_mean,count_years
0,Івано-Франківська область,EIT,144.559473,0.368883,1.003596,3
1,Івано-Франківська область,ZNO,145.913723,0.476189,1.010503,3
2,Волинська область,EIT,145.800046,0.528326,1.012214,3
3,Волинська область,ZNO,146.013451,0.493993,1.011182,3
4,Вінницька область,EIT,143.778823,0.277182,0.998307,3
5,Вінницька область,ZNO,143.403667,0.266280,0.993115,3
6,Дніпропетровська область,EIT,142.629507,0.133678,0.990367,3
7,Дніпропетровська область,ZNO,143.307449,0.261358,0.992449,3
8,Донецька область,ZNO,143.179268,0.252377,0.991546,3
9,Житомирська область,EIT,142.693665,0.137109,0.990743,3


In [81]:
import altair as alt
import pandas as pd
import numpy as np

# 1. Завантажуємо геодані України (регіони) через topojson
ukraine = alt.topo_feature(
    "https://raw.githubusercontent.com/org-scn-design-studio-community/sdkcommunitymaps/master/geojson/Europe/Ukraine-regions.json",
    'UKR_adm1'
)

# 2. Маапінг українських назв регіонів до англійських назв у geojson
mapping = {
    "Черкаська область": "Cherkasy",
    "Чернігівська область": "Chernihiv",
    "Чернівецька область": "Chernivtsi",
    "Дніпропетровська область": "Dnipropetrovs'k",
    "Донецька область": "Donets'k",
    "Івано-Франківська область": "Ivano-Frankivs'k",
    "Харківська область": "Kharkiv",
    "Херсонська область": "Kherson",
    "Хмельницька область": "Khmel'nyts'kyy",
    "Київська область": "Kiev",
    "Кіровоградська область": "Kirovohrad",
    "Луганська область": "Luhans'k",
    "Львівська область": "L'viv",
    "Миколаївська область": "Mykolayiv",
    "Одеська область": "Odessa",
    "Полтавська область": "Poltava",
    "Рівненська область": "Rivne",
    "Сумська область": "Sumy",
    "Тернопільська область": "Ternopil'",
    "Вінницька область": "Vinnytsya",
    "Волинська область": "Volyn",
    "Закарпатська область": "Transcarpathia",
    "Запорізька область": "Zaporizhzhya",
    "Житомирська область": "Zhytomyr",
    "м.Київ": "Kiev City"
}

# 3. Підготовка даних для карти
# Тут припускаємо, що у тебе є DataFrame `data` з колонками:
# 'region_name' (укр), 'score_norm', 'year'

# Зведемо по роках до двох періодів: до 2022 і з 2022 року
before = data[data['year'] < 2022].groupby('region_name').agg(
    mean_score=('score_norm', 'mean'),
    count=('score_norm', 'count')
).reset_index()

after = data[data['year'] >= 2022].groupby('region_name').agg(
    mean_score=('score_norm', 'mean'),
    count=('score_norm', 'count')
).reset_index()

# Об'єднуємо для порівняння
compare = before.merge(after, on='region_name', suffixes=('_before', '_after'))

# Обчислення зміни із врахуванням ваг (кількості записів)
compare['change'] = (
    compare['mean_score_after'] * compare['count_after'] -
    compare['mean_score_before'] * compare['count_before']
) / (compare['count_before'] + compare['count_after'])

# Мапінг до англійських назв для topojson
compare['EORegName'] = compare['region_name'].map(mapping)

# Додаємо Крим вручну, щоб не було помилок
missing = pd.DataFrame({
    'EORegName': ['Crimea'],
    'change': [np.nan]
})

# Формуємо фінальний DataFrame для карти
df_for_map = pd.concat([compare[['EORegName', 'change']], missing], ignore_index=True)

# 4. Визначаємо області без заливки (білими)
excluded_regions = ["Crimea", "Luhans'k", "Donets'k", "Kherson"]

# 5. Функція побудови карти
def ukraine_map(df):
    base_map = (alt.Chart(ukraine).mark_geoshape(
            stroke='black', strokeWidth=1
        ).encode(
            color=alt.Color("change:Q", 
                            scale=alt.Scale(scheme='redyellowgreen', domainMid=0),
                            title="Change in Score"),
            tooltip=[
                alt.Tooltip("properties.NAME_1:N", title="Region"),
                alt.Tooltip("change:Q", title="Change")
            ]
        ).transform_lookup(
            lookup="properties.NAME_1",
            from_=alt.LookupData(df, key="EORegName", fields=["EORegName", "change"])
        )
        # Виключаємо області без заливки з основної заливки
        .transform_filter(~alt.FieldOneOfPredicate(field='properties.NAME_1', oneOf=excluded_regions))
    )

    excluded_outline = (
        alt.Chart(ukraine).mark_geoshape(
            fill='white',  # Білий фон (без заливки)
            stroke='black',
            strokeWidth=1
        ).transform_filter(
            alt.FieldOneOfPredicate(field='properties.NAME_1', oneOf=excluded_regions)
        )
    )

    final_map = (base_map + excluded_outline).properties(
        width=800,
        height=600,
        title="Changes normalized score in EIT comparing to the NMT"
    )

    return final_map

# 6. Відображаємо карту
ukraine_map(df_for_map)
chart = ukraine_map(df_for_map)

chart.save('ukraine_map.html')


In [75]:
import altair as alt
import pandas as pd
import numpy as np

# 1. Завантажуємо геодані України (регіони) через topojson
ukraine = alt.topo_feature(
    "https://raw.githubusercontent.com/org-scn-design-studio-community/sdkcommunitymaps/master/geojson/Europe/Ukraine-regions.json",
    'UKR_adm1'
)

# 2. Маапінг українських назв регіонів до англійських назв у geojson
mapping = {
    "Черкаська область": "Cherkasy",
    "Чернігівська область": "Chernihiv",
    "Чернівецька область": "Chernivtsi",
    "Дніпропетровська область": "Dnipropetrovs'k",
    "Донецька область": "Donets'k",
    "Івано-Франківська область": "Ivano-Frankivs'k",
    "Харківська область": "Kharkiv",
    "Херсонська область": "Kherson",
    "Хмельницька область": "Khmel'nyts'kyy",
    "Київська область": "Kiev",
    "Кіровоградська область": "Kirovohrad",
    "Луганська область": "Luhans'k",
    "Львівська область": "L'viv",
    "Миколаївська область": "Mykolayiv",
    "Одеська область": "Odessa",
    "Полтавська область": "Poltava",
    "Рівненська область": "Rivne",
    "Сумська область": "Sumy",
    "Тернопільська область": "Ternopil'",
    "Вінницька область": "Vinnytsya",
    "Волинська область": "Volyn",
    "Закарпатська область": "Transcarpathia",
    "Запорізька область": "Zaporizhzhya",
    "Житомирська область": "Zhytomyr",
    "м.Київ": "Kiev City"
}

# 3. Підготовка даних для карти
# Тут припускаємо, що у тебе є DataFrame `data` з колонками:
# 'region_name' (укр), 'score_norm', 'year'

# Зведемо по роках до двох періодів: до 2022 і з 2022 року
before = data[data['year'] < 2022].groupby('region_name').agg(
    mean_score=('score_rel', 'mean'),
    count=('score_rel', 'count')
).reset_index()

after = data[data['year'] >= 2022].groupby('region_name').agg(
    mean_score=('score_rel', 'mean'),
    count=('score_rel', 'count')
).reset_index()

# Об'єднуємо для порівняння
compare = before.merge(after, on='region_name', suffixes=('_before', '_after'))

# Обчислення зміни із врахуванням ваг (кількості записів)
compare['change'] = (
    compare['mean_score_after'] * compare['count_after'] -
    compare['mean_score_before'] * compare['count_before']
) / (compare['count_before'] + compare['count_after'])

# Мапінг до англійських назв для topojson
compare['EORegName'] = compare['region_name'].map(mapping)

# Додаємо Крим вручну, щоб не було помилок
missing = pd.DataFrame({
    'EORegName': ['Crimea'],
    'change': [np.nan]
})

# Формуємо фінальний DataFrame для карти
df_for_map = pd.concat([compare[['EORegName', 'change']], missing], ignore_index=True)

# 4. Визначаємо області без заливки (білими)
excluded_regions = ["Crimea", "Luhans'k", "Donets'k", "Kherson"]

# 5. Функція побудови карти
def ukraine_map(df):
    base_map = (alt.Chart(ukraine).mark_geoshape(
            stroke='black', strokeWidth=1
        ).encode(
            color=alt.Color("change:Q", 
                            scale=alt.Scale(scheme='redyellowgreen', domainMid=0),
                            title="Change in Score"),
            tooltip=[
                alt.Tooltip("properties.NAME_1:N", title="Region"),
                alt.Tooltip("change:Q", title="Change")
            ]
        ).transform_lookup(
            lookup="properties.NAME_1",
            from_=alt.LookupData(df, key="EORegName", fields=["EORegName", "change"])
        )
        # Виключаємо області без заливки з основної заливки
        .transform_filter(~alt.FieldOneOfPredicate(field='properties.NAME_1', oneOf=excluded_regions))
    )

    excluded_outline = (
        alt.Chart(ukraine).mark_geoshape(
            fill='white',  # Білий фон (без заливки)
            stroke='black',
            strokeWidth=1
        ).transform_filter(
            alt.FieldOneOfPredicate(field='properties.NAME_1', oneOf=excluded_regions)
        )
    )

    final_map = (base_map + excluded_outline).properties(
        width=800,
        height=600,
        title="Changes normalized score in EIT comparing to the NMT"
    )

    return final_map

# 6. Відображаємо карту
ukraine_map(df_for_map)


alt.LayerChart(...)

In [74]:
data

,region_name,average_score,year,source,score_norm,score_rel
0,Івано-Франківська область,149.334603,2020,ZNO,0.578887,1.019871
1,Волинська область,148.794666,2020,ZNO,0.531764,1.016183
2,Вінницька область,144.970217,2020,ZNO,0.197989,0.990065
3,Дніпропетровська область,145.102046,2020,ZNO,0.209494,0.990965
4,Донецька область,143.967670,2020,ZNO,0.110493,0.983218
...,...,...,...,...,...,...
136,Хмельницька область,139.375244,2024,EIT,0.285166,1.002219
137,Черкаська область,138.331308,2024,EIT,0.147698,0.994712
138,Чернівецька область,138.119851,2024,EIT,0.119853,0.993192
139,Чернігівська область,138.846084,2024,EIT,0.215485,0.998414


In [43]:
import pandas as pd
import os
import re

folder_path = 'filtered_data'
dfs = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        year_match = re.search(r'(\d{4})', file)
        if not year_match:
            continue
        year = int(year_match.group(1))
        
        source = 'ZNO' if year < 2022 else 'EIT'
        
        path = os.path.join(folder_path, file)
        df = pd.read_csv(path)

        if 'region_name' in df.columns and 'average_score' in df.columns:
            grouped = df.groupby('region_name', as_index=False)['average_score'].count()
            grouped = grouped.rename(columns={'average_score': 'count'})
            grouped['year'] = year
            grouped['source'] = source
            dfs.append(grouped)

data = pd.concat(dfs, ignore_index=True)
print(data.head())


/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/3577932667.py:18: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/3577932667.py:18: DtypeWarning: Columns (117,118,121,123,124,125,126,137,138,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/3577932667.py:18: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/3577932667.py:18: DtypeWarning: Columns (100,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


                 region_name  count  year source
0  Івано-Франківська область   7007  2020    ZNO
1          Волинська область   6063  2020    ZNO
2          Вінницька область   8521  2020    ZNO
3   Дніпропетровська область  16493  2020    ZNO
4           Донецька область   8051  2020    ZNO


In [44]:
data

,region_name,count,year,source
0,Івано-Франківська область,7007,2020,ZNO
1,Волинська область,6063,2020,ZNO
2,Вінницька область,8521,2020,ZNO
3,Дніпропетровська область,16493,2020,ZNO
4,Донецька область,8051,2020,ZNO
...,...,...,...,...
147,Хмельницька область,9955,2024,EIT
148,Черкаська область,8458,2024,EIT
149,Чернівецька область,6060,2024,EIT
150,Чернігівська область,6557,2024,EIT


In [66]:
df = pd.read_csv("filtered_data/2020.csv")
# count_by_region = df.groupby('region_name').size().reset_index(name='count')
# 
# print(count_by_region)
df

/var/folders/5r/4d3mqngx3c3bynq7lh4_h3tr0000gn/T/ipykernel_16227/1942552290.py:1: DtypeWarning: Columns (96,97,100,102,103,104,105,116,117,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("filtered_data/2020.csv")


,id,birth_year,gender,region_name,area_name,territory_name,region_type,territory_type,class_profile,class_language,...,spanish_score,spanish_pt_name,spanish_pt_region,spanish_pt_area,spanish_pt_territory,student_age,region_flag,subjects_count,total_score,average_score
0,1f6c1a4f-8c8a-45c5-bc6c-0eaaeb3a2c30,2003,чоловіча,Донецька область,м.Маріуполь,Приморський район міста,випускник закладу загальної середньої освіти 2...,місто,Історичний,російська,...,NaN,NaN,NaN,NaN,NaN,17,east,4,437.0,109.250000
1,4ba3e361-8e8c-48c1-830d-1ea85b9d4c77,2003,чоловіча,Полтавська область,Миргородський район,с.Хомутець,випускник закладу загальної середньої освіти 2...,село,Української філології,українська,...,NaN,NaN,NaN,NaN,NaN,17,central,3,421.0,140.333333
2,a7d7800a-e850-4307-b79a-98d96924ef05,2002,чоловіча,м.Київ,м.Київ,Святошинський район міста,випускник закладу загальної середньої освіти 2...,місто,Універсальний,українська,...,NaN,NaN,NaN,NaN,NaN,18,other,4,514.0,128.500000
3,e73ff63a-9fd7-43dd-86d7-c5c53ba6e865,2003,чоловіча,Сумська область,Сумська область,м.Конотоп,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,NaN,NaN,NaN,NaN,17,north,3,387.0,129.000000
4,0b95a736-7ad9-49db-854a-abdb959f02f4,2003,жіноча,Львівська область,м.Львів,Галицький район міста,випускник закладу загальної середньої освіти 2...,місто,Універсальний,українська,...,NaN,NaN,NaN,NaN,NaN,17,west,4,602.0,150.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201207,b9cb08b0-e084-488c-85ee-b8da94de04f5,2003,чоловіча,Чернігівська область,Чернігівська область,м.Прилуки,випускник закладу загальної середньої освіти 2...,місто,Інформаційно-технологічний,українська,...,NaN,NaN,NaN,NaN,NaN,17,north,4,709.0,177.250000
201208,7d3d693a-09b8-48f9-8c6e-01671aafcdca,2003,жіноча,Одеська область,м.Одеса,Приморський район міста,Студент закладу вищої освіти,місто,Молодший спеціаліст,українська,...,NaN,NaN,NaN,NaN,NaN,17,south,3,381.0,127.000000
201209,4906881e-b034-440d-ac42-6df966fa8077,2003,чоловіча,Івано-Франківська область,Івано-Франківська область,м.Калуш,випускник закладу загальної середньої освіти 2...,місто,Історичний,українська,...,NaN,NaN,NaN,NaN,NaN,17,west,3,593.5,197.833333
201210,90437a9b-94cc-4e26-8390-afdd987c0958,2003,чоловіча,Полтавська область,Полтавська область,м.Лубни,випускник закладу загальної середньої освіти 2...,місто,Фізико-математичний,українська,...,NaN,NaN,NaN,NaN,NaN,17,central,3,407.0,135.666667
